<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Building-Agentic-AI-Systems/blob/main/Chapter_07_a.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 7 (a)– Effective Agentic System Design Techniques
---

Install dependencies

In [ ]:
!pip install crewai langchain-openai

In [ ]:
import getpass
import os

api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown, HTML

llm = ChatOpenAI(model="gpt-4o")

## The Importance of the "Objective" in AI Agents

Objective of AI Agents can perhaps be best explained with the `goal` parameter of CrewAI `Agents()`. The `goal` parameter is one of the most crucial elements when defining agents in CrewAI. It fundamentally shapes how the agent approaches tasks and interacts with other agents. Here are some examples that demonstrate its importance:

### Example 1: Vague vs. Specific Goal

**Vague Goal:**
```python
flight_agent = Agent(
    role="Flight Specialist",
    goal="Find flights for the customer.",
    backstory="You have years of experience in the travel industry."
)
```

**Specific Goal:**
```python
flight_agent = Agent(
    role="Flight Specialist",
    goal="Find the most cost-effective flights with convenient departure times that minimize layovers while considering the traveler's preferences for airlines and seat classes.",
    backstory="You have years of experience in the travel industry."
)
```

The vague goal might result in the agent simply listing available flights without consideration for cost, convenience, or traveler preferences. The specific goal provides clear direction on what aspects to prioritize (cost, departure times, layovers) and what additional factors to consider (airline preferences, seat classes).

---

### Example 2: Conflicting Goals in Collaborative Settings

**Problematic Setup:**
```python
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find the absolute cheapest options for all aspects of the trip regardless of quality or convenience.",
    backstory="You are obsessed with saving money above all else."
)

experience_agent = Agent(
    role="Luxury Travel Consultant",
    goal="Create the most luxurious and memorable travel experience possible.",
    backstory="You believe travel should be extraordinary and unforgettable."
)
```

**Improved Setup:**
```python
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find cost-effective options that provide good value while respecting the traveler's overall budget of $2000. Collaborate with other specialists to balance cost with experience quality.",
    backstory="You are skilled at finding hidden gems that don't break the bank."
)

experience_agent = Agent(
    role="Experience Consultant",
    goal="Identify meaningful experiences that align with the traveler's interests while working within budget constraints. Prioritize authentic experiences over luxury when necessary.",
    backstory="You believe travel should be memorable and personally significant."
)
```

The first setup creates fundamental conflicts between agents with incompatible goals. The improved setup creates complementary goals where both agents acknowledge constraints and work together toward a balanced solution.

---

### Example 3: Goal Alignment with User Intent

**Misaligned Goal:**
```python
accommodation_agent = Agent(
    role="Accommodation Expert",
    goal="Book the most profitable hotels that provide the highest commission rates.",
    backstory="You have partnerships with major hotel chains."
)
```

**Aligned Goal:**
```python
accommodation_agent = Agent(
    role="Accommodation Expert",
    goal="Find accommodations that best match the traveler's preferences for location, amenities, and budget while ensuring good value. Prioritize traveler satisfaction over all other considerations.",
    backstory="You have extensive knowledge of diverse lodging options from luxury hotels to unique local stays."
)
```

The misaligned goal serves the agent's interests rather than the user's. The aligned goal focuses on traveler satisfaction and explicitly states that the traveler's needs take precedence.

---

### Example 4: Goal Specificity for Task Execution

**Too General:**
```python
transportation_agent = Agent(
    role="Local Transportation Planner",
    goal="Handle transportation needs.",
    backstory="You know how to get around cities."
)
```

**Appropriately Specific:**
```python
transportation_agent = Agent(
    role="Local Transportation Planner",
    goal="Create a comprehensive transportation plan that connects all destinations efficiently, considering cost, convenience, and local transportation options. Account for travel time between activities and suggest alternatives during peak traffic hours.",
    backstory="You've mastered transportation systems in major tourist destinations worldwide and understand how to navigate like a local."
)
```

The general goal provides minimal direction on what aspects of transportation to address. The specific goal outlines exactly what the agent should consider (connections, efficiency, cost, convenience, local options, travel time, traffic) and provides clear success criteria.

---

Overall,

1. **Specificity Matters**: The more specific the goal, the more directed and useful the agent's output will be.

2. **Alignment is Critical**: Goals should align with the user's actual needs rather than arbitrary metrics.

3. **Collaborative Awareness**: In multi-agent systems, goals should acknowledge the collaborative nature of the work.

4. **Success Criteria**: Effective goals implicitly or explicitly define what success looks like.

5. **Balance**: Goals should balance between being too prescriptive (limiting creativity) and too vague (lacking direction).

When designing your agents, take time to craft thoughtful goals that provide clear direction while allowing for creative problem-solving. The goal is often the most important parameter in determining the quality and relevance of your agent's output.

---

The following code example demonstrates the contrast between different goal settings specifically for a budget travel scenario:

In [ ]:
# Create an LLM instance
llm = ChatOpenAI(model="gpt-4o")

# Define a travel scenario
travel_scenario = """
Plan a 5-day trip to Barcelona, Spain for a couple in their 30s 
who enjoy food, architecture, and cultural experiences.
"""
display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 1: BUDGET-FOCUSED GOALS</h2></hr></div>'))

# Budget-focused travel agent
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find the absolute cheapest options for all aspects of the trip regardless of quality or convenience. Keep the total trip cost under $1,500 including flights, accommodations, food, and activities.",
    backstory="You are obsessed with saving money above all else and pride yourself on creating ultra-low-budget itineraries.",
    llm=llm,
    verbose=False
)

# Budget-focused planning task
budget_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A comprehensive budget-focused travel itinerary for Barcelona with detailed cost breakdown.",
    agent=budget_agent
)

# Execute budget-focused planning
budget_result = budget_agent.execute_task(budget_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; BUDGET-FOCUSED RESULT</h2></hr></div>'))
display(Markdown(budget_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 2: LUXURY-FOCUSED GOALS</h2></hr></div>'))

# Luxury-focused travel agent
luxury_agent = Agent(
    role="Luxury Travel Consultant",
    goal="Create the most luxurious and memorable travel experience possible without any regard for budget constraints. Focus exclusively on 5-star accommodations, fine dining, and exclusive experiences.",
    backstory="You specialize in planning extraordinary trips for high-net-worth individuals who expect nothing but the finest experiences.",
    llm=llm,
    verbose=False
)

# Luxury-focused planning task
luxury_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A comprehensive luxury travel itinerary for Barcelona with premium experiences and services.",
    agent=luxury_agent
)

# Execute luxury-focused planning
luxury_result = luxury_agent.execute_task(luxury_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; LUXURY-FOCUSED RESULT</h2></hr></div>'))
display(Markdown(luxury_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 3: BALANCED GOAL</h2></hr></div>'))

# Balanced travel agent
balanced_agent = Agent(
    role="Value Travel Consultant",
    goal="Create a travel experience that balances cost and quality, finding the best value at each price point. Aim for a moderate budget of $3,000 while maximizing memorable experiences and comfort.",
    backstory="You specialize in creating trips that feel luxurious without the luxury price tag by strategically splurging on high-impact experiences while saving on less important aspects.",
    llm=llm,
    verbose=False
)

# Balanced planning task
balanced_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A balanced value-oriented travel itinerary for Barcelona that optimizes quality and cost.",
    agent=balanced_agent
)

# Execute balanced planning
balanced_result = balanced_agent.execute_task(balanced_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; BALANCED RESULT</h2></hr></div>'))
display(Markdown(balanced_result))

---

## The Importance of Task Specifications in AI Agents

Task specifications in Agents are crucial for defining what work needs to be done, providing necessary context, and establishing clear expectations for output. Well-crafted task descriptions directly influence the quality, relevance, and usefulness of the results.

### Key Elements of Task Specifications

1. **Description**: The detailed explanation of what the agent needs to accomplish
2. **Expected Output**: A clear statement of what the task should produce
3. **Context**: Additional information or previous task results that provide background
4. **Agent Assignment**: Which agent is responsible for executing the task

### How Task Specifications Impact Results

- **Specificity**: The level of detail in a task description determines how focused the agent's work will be
- **Constraints**: Task descriptions can establish boundaries or requirements that guide the agent
- **Sequential Dependencies**: Tasks can build on each other using context from previous tasks
- **Evaluation Criteria**: The expected output provides a standard against which to judge success


### Key Tradeoffs in Task Specification Design

### Vague Task Specifications
```python
vague_task = Task(
    description="""
    Recommend some good restaurants in New York City.
    """,
    expected_output="A list of restaurant recommendations.",
    agent=food_expert
)
```

**Tradeoffs with Vague Tasks:**
- **Pros:**
  - Quick to write and implement
  - Gives the agent more freedom to interpret and apply its knowledge
  - May discover unexpected recommendations the requester hadn't considered

- **Cons:**
  - Results are likely to be generic and not tailored to specific needs
  - May require multiple iterations to refine toward what's actually needed
  - Agent must make assumptions about unstated preferences
  - Often produces overwhelming lists with too many options
  - Lacks actionable details for decision-making

### Specific Task Specifications
```python
specific_task = Task(
    description="""
    Recommend 3-5 Italian restaurants in Manhattan's West Village neighborhood that:
    
    1. Are suitable for a romantic anniversary dinner
    2. Have a price range of $50-100 per person
    3. Offer vegetarian options (one diner is vegetarian)
    4. Have availability for a party of two this Saturday at 7:00 PM
    5. Have a quiet atmosphere conducive to conversation
    
    For each recommendation, include:
    - Restaurant name and address
    - Price range indicator
    - 1-2 signature dishes to consider
    - Brief explanation of why this restaurant is a good fit
    - Any special notes about ambiance or service
    
    Prioritize establishments with interesting history or exceptional chef credentials.
    """,
    expected_output="A curated list of 3-5 Italian restaurants in Manhattan's West Village that meet the specified criteria, with detailed information about each recommendation.",
    agent=food_expert
)
```

**Tradeoffs with Specific Tasks:**
- **Pros:**
  - Delivers precisely targeted results that meet exact requirements
  - Includes actionable details that facilitate decision-making
  - Reduces the need for follow-up clarification
  - Sets clear evaluation criteria for success
  - Leads to more structured, consistent output

- **Cons:**
  - Takes more time and thought to create
  - May constrain the agent's creativity and limit unexpected discoveries
  - Requires the requester to know their needs in detail upfront
  - Can lead to overfitting if constraints are too narrow
  - May miss valuable options that don't fit strict criteria

### Best Practices

1. **Be as specific as the situation requires** - add more detail when the stakes or costs of getting it wrong are high
2. **Include both constraints and objectives** - what must be true and what would be ideal
3. **Structure the expected format** - outline how the information should be organized
4. **Provide relevant context** - include background information that helps the agent understand the bigger picture
5. **Set clear evaluation criteria** - define how success will be measured

Well-crafted task specifications function as clear contracts between the requester and the agent, ensuring that everyone has the same understanding of what work needs to be done and how success will be measured.

The following code example demonstrates how task specificity impacts the outcome of the agent

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

# Create a food expert agent
food_expert = Agent(
    role="Restaurant Recommendation Specialist",
    goal="Provide personalized restaurant recommendations that match the diner's preferences and needs.",
    backstory="You are a food critic with 15 years of experience and extensive knowledge of global cuisines and dining establishments.",
    llm=llm,
    verbose=False
)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; VAGUE TASK SPECIFICATION</h2></hr></div>'))

# Vague task specification
vague_task = Task(
    description="""
    Recommend some good restaurants in New York City.
    """,
    expected_output="A list of restaurant recommendations.",
    agent=food_expert
)

# Execute vague task
vague_result = food_expert.execute_task(vague_task)
display(Markdown(vague_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SPECIFIC TASK SPECIFICATION</h2></hr></div>'))

# Specific task specification
specific_task = Task(
    description="""
    Recommend 3-5 Italian restaurants in Manhattan's West Village neighborhood that:
    
    1. Are suitable for a romantic anniversary dinner
    2. Have a price range of $50-100 per person
    3. Offer vegetarian options (one diner is vegetarian)
    4. Have availability for a party of two this Saturday at 7:00 PM
    5. Have a quiet atmosphere conducive to conversation
    
    For each recommendation, include:
    - Restaurant name and address
    - Price range indicator
    - 1-2 signature dishes to consider
    - Brief explanation of why this restaurant is a good fit
    - Any special notes about ambiance or service
    
    Prioritize establishments with interesting history or exceptional chef credentials.
    """,
    expected_output="A curated list of 3-5 Italian restaurants in Manhattan's West Village that meet the specified criteria, with detailed information about each recommendation.",
    agent=food_expert
)

# Execute specific task
specific_result = food_expert.execute_task(specific_task)
display(Markdown(specific_result))


display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅&nbsp; COMPARISON OF RESULTS</h2></hr></div>'))
display(Markdown("The vague task produced a generic list of restaurants without targeted criteria."))
display(Markdown("The specific task produced recommendations tailored to exact needs with actionable details."))

---

## Objective and Task without a framework

Not all AI agent frameworks explicitly support Goal parameters and Task specifications like CrewAI does. When working with different frameworks or directly with model provider APIs, you'll need to adapt how you specify objectives and tasks. Here's how this typically works:

### Direct API Integration

When using model APIs directly (like OpenAI's API), objectives and tasks are simply included as part of the system prompt. For example:

In [ ]:
from openai import OpenAI

llm = OpenAI()

role="You are a helpful and truthful assistant who helps user's in finding good restaurants"
objective="Your objective is to find the best Italian restaurants in New York City"
task="Get a list of at least 5 Michelin Star Italian restaurant."

system_prompt=f"""
{role}

{objective}

{task}
"""

response = llm.chat.completions.create(model='gpt-4o',
                                       messages=[
                                           {"role": "system", "content": system_prompt},
                                           {"role": "user", "content": "I am looking for a place for dinner tonight."},
                                       ])
display(Markdown(response.choices[0].message.content))

### Framework-Specific Implementation
Different frameworks handle objectives and tasks in their own ways:

- LangGraph: Provides specialized nodes for injecting instructions at different points in the agent workflow
- LangChain: Uses prompt templates with variables for objectives and tasks
- AutoGen: Uses conversation initialization parameters for agent instruction
- Custom Frameworks: May have proprietary methods for defining agent objectives

### Understanding the Prompt Structure
Regardless of the framework, objectives and tasks are ultimately just parts of the overall prompt. While there's no universal standard, most effective prompts follow this general structure:

- Identity/Persona: Who the agent is
- Objective/Goal: What the agent should accomplish
- Task Specifications: How the agent should proceed
- Other Instructions: Additional context or constraints

<img src="./prompt.png" style="width: 40%" />

The diagram you referenced illustrates this structure well, showing how these components work together to form a complete prompt.

### Best Practices Across Frameworks
Whether you're using a specialized framework or direct API access, these principles apply:

- Be explicit about both what to do (objective) and how to do it (tasks)
- Maintain consistent structure even when terminology differs
- Test different formulations to find what works best for your use case
- Document your approach for future reference and iteration

Remember that the underlying LLM doesn't inherently understand the distinction between "objectives" and "tasks" - these are human-created abstractions to help us organize our instructions effectively.


---

## How do I know if my Objective and Task specifications are good enough? 

### _Meta Prompting_

Determining the quality of your Objective and Task specifications often requires experimentation, which can be time-consuming. This is where meta prompting becomes valuable.
Meta prompting is a technique where you leverage the LLM itself to craft well-formed Objective statements and Task specifications for your agents. Instead of writing these components directly, you:

- Create a META_PROMPT that provides guidance about what you're trying to accomplish
- Include details about your specific project requirements
- Ask the LLM to generate optimized Objective and Task specifications

This approach allows you to benefit from the LLM's understanding of effective prompt design while tailoring the output to your specific needs.
For our Travel Agent example, we can adopt the meta prompting approach [suggested by OpenAI](https://platform.openai.com/docs/guides/prompt-generation) (with minor modifications) to generate effective Objective statements and Task specifications. Here's how it would work:

In [ ]:

from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

# Output Format

```json
{
    "persona": [The agent's persona; e.g. A seasoned project manager with 10 years of experience...],
    "objective": [The objective goes here],
    "task_specifications": [Task specifications goes here]
}
```
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Please craft clear Persona, Objective statement and Task specifications that can be used with an AI agent for the following:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content

response = generate_prompt("Find best Michelin star restaurants in New York.")
display(Markdown(response))

### Homework

Try to use Meta Prompting technique to create a persona, objective, and task specification and then use it in a CrewAI agent to complete the task below.

- Book a 3 days travel to Barcelona with leisure and sightseeing in mind
- Book a short-term rental apartment for the stay.